### Seq2Seq모델 빌드를 위한 데이터 전처리

In [1]:
import os
import re
import json
import numpy as np
import pandas as pd

from konlpy.tag import Okt

In [2]:
PATH = "./Chatbot_data/ChatbotData.csv"
VOCAB_PATH = "./Chatbot_data/vocabulary.txt"

In [3]:
# 데이터 읽어오는 함수 정의
def load_data(path: str):
    data_df = pd.read_csv(path, header=0, sep=",")[:100]
    question, answer = data_df['Q'].to_list(), data_df['A'].to_list()
    return question, answer

In [4]:
# 텍스트에 대해 불필요한 문자들을 제거하고 split한다.
def word_tokenizer(text_list: list) -> list:
    vocab = []
    
    for text in text_list:
        filter = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]" # filter 정의
        CHANGE_FILTER = re.compile(filter)

        cleaned_text = re.sub(CHANGE_FILTER, "", text) # filter에 부합하는 문자들을 제거하고
        text_list = cleaned_text.split() # 공백을 기준으로 리스트화 하고
        vocab.extend(text_list) # 사전에 추가한다.
    
    return vocab

In [5]:
# 형태소 분석 후 리스트에 담는다
def preprocess_like_morphlized(data: list):
    morph_analyzer = Okt()
    result_data = []
    
    for seq in data:
        morphed_data = " ".join(morph_analyzer.morphs(seq))
        result_data.append(morphed_data)
        
    return result_data

In [6]:
# 정수형 인코딩 된 형태의 사전을 리턴한다
def make_vocabulary(vocabulary_list: list):
    char2idx = {char : i for i, char in enumerate(vocabulary_list)}
    idx2char = {i : char for i, char in enumerate(vocabulary_list)}
    
    return char2idx, idx2char

In [7]:
inputs, outputs = load_data(PATH)

data = inputs + outputs
vocab2 = list(set(word_tokenizer(preprocess_like_morphlized(data)))) # 중복제거

In [8]:
PAD = "<PAD>" # padding token
SOS = "<SOS>" # start of sentence token
EOS = "<EOS>" # end of sentence token
OOV = "<OOV>" # out of vocabulary token

PAD_IDX = 0 # padding token index
SOS_IDX = 1 # start of sentence token index
EOS_IDX = 2 # end of sentence token index
OOV_IDX = 3 # out of vocabulary token index

# insert into vocabulary
vocab2.insert(PAD_IDX, PAD)
vocab2.insert(SOS_IDX, SOS)
vocab2.insert(EOS_IDX, EOS)
vocab2.insert(OOV_IDX, OOV)

In [9]:
with open(VOCAB_PATH, "w", encoding="utf-8") as f:
    for word in vocab2:
        f.write(word + "\n")

In [10]:
vocab_list = []

if os.path.exists(VOCAB_PATH):
    with open(VOCAB_PATH, "r", encoding="utf-8") as f:
        for vocab in f.readlines():
            vocab_list.append(vocab.replace("\n", ""))
else:
    print(f"Cannot read {VOCAB_PATH} file")

In [11]:
char2idx, idx2char = make_vocabulary(vocab_list)

In [12]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<OOV>': 3,
 '오개': 4,
 '컴퓨터': 5,
 '짝녀일': 6,
 '막막': 7,
 '탈': 8,
 '않았을': 9,
 '해외': 10,
 '춤': 11,
 '용기': 12,
 '몸값': 13,
 '모르니': 14,
 '된다는': 15,
 '놓치지': 16,
 '걸리려나': 17,
 '나왔네': 18,
 '흐려지기': 19,
 '뜻': 20,
 '나온거': 21,
 '빌리면': 22,
 '내길인가': 23,
 '분명': 24,
 '느껴져': 25,
 '깔끔히': 26,
 '모르지요': 27,
 '만들까': 28,
 '해심': 29,
 '아파하면서': 30,
 '담하더니': 31,
 '해주신': 32,
 '켜면서': 33,
 '이었겠네요': 34,
 '됐나': 35,
 '냉정하나': 36,
 '되진': 37,
 '연애스타일': 38,
 '서툴어요': 39,
 '지침': 40,
 '육체': 41,
 '잊게': 42,
 '우울합니다': 43,
 '같다가도': 44,
 '리얼': 45,
 '싫다': 46,
 '띠': 47,
 '탈모': 48,
 '나쁘지는': 49,
 '외롭다고': 50,
 '트인': 51,
 '런가': 52,
 '졌는데': 53,
 '수천': 54,
 '땀': 55,
 '찾으려고': 56,
 '냄': 57,
 '좋아해도': 58,
 '듣니': 59,
 '들이': 60,
 '놓으렵니다': 61,
 '들켰는데': 62,
 '쌍': 63,
 '이야기': 64,
 '몇번': 65,
 '갑작스럽겠네요': 66,
 '직장인': 67,
 '아니길': 68,
 '나아질거예요': 69,
 '풀어야겠다': 70,
 '착': 71,
 '돋을': 72,
 '청결한': 73,
 '가상': 74,
 '종지부': 75,
 '따라와주지': 76,
 '어떡하냐': 77,
 '놓느라': 78,
 '재밌다': 79,
 '땀나네': 80,
 '뜨고': 81,
 '내야': 82,
 '무기력증': 83,

In [13]:
idx2char

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<OOV>',
 4: '오개',
 5: '컴퓨터',
 6: '짝녀일',
 7: '막막',
 8: '탈',
 9: '않았을',
 10: '해외',
 11: '춤',
 12: '용기',
 13: '몸값',
 14: '모르니',
 15: '된다는',
 16: '놓치지',
 17: '걸리려나',
 18: '나왔네',
 19: '흐려지기',
 20: '뜻',
 21: '나온거',
 22: '빌리면',
 23: '내길인가',
 24: '분명',
 25: '느껴져',
 26: '깔끔히',
 27: '모르지요',
 28: '만들까',
 29: '해심',
 30: '아파하면서',
 31: '담하더니',
 32: '해주신',
 33: '켜면서',
 34: '이었겠네요',
 35: '됐나',
 36: '냉정하나',
 37: '되진',
 38: '연애스타일',
 39: '서툴어요',
 40: '지침',
 41: '육체',
 42: '잊게',
 43: '우울합니다',
 44: '같다가도',
 45: '리얼',
 46: '싫다',
 47: '띠',
 48: '탈모',
 49: '나쁘지는',
 50: '외롭다고',
 51: '트인',
 52: '런가',
 53: '졌는데',
 54: '수천',
 55: '땀',
 56: '찾으려고',
 57: '냄',
 58: '좋아해도',
 59: '듣니',
 60: '들이',
 61: '놓으렵니다',
 62: '들켰는데',
 63: '쌍',
 64: '이야기',
 65: '몇번',
 66: '갑작스럽겠네요',
 67: '직장인',
 68: '아니길',
 69: '나아질거예요',
 70: '풀어야겠다',
 71: '착',
 72: '돋을',
 73: '청결한',
 74: '가상',
 75: '종지부',
 76: '따라와주지',
 77: '어떡하냐',
 78: '놓느라',
 79: '재밌다',
 80: '땀나네',
 81: '뜨고',
 82: '내야',
 83: '무기력증',

### 인코더, 디코더 용 데이터 구성

In [14]:
filter = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"
CHANGE_FILTER = re.compile(filter)
MAX_LEN = 76

def make_sequence(value, dictionary, tokenize_as_morph = False, padding = "pre", position = ""):
    if not tokenize_as_morph: # value 안의 각 문장들이 형태소 분석이 되어있지 않은 상태라면
        value = preprocess_like_morphlized(value) # 위에서 정의해둔 함수로 형태소 분석을 진행한다.
    
    padded_sequence = [] # 패딩된 각 문장들을 담을 빈 리스트
    for seq in value: # value안의 각 문장들을 순회하면서
        temp_list = [] # zero padding, 인코딩된 문장 리스트를 담을 빈 리스트
        seq_idx = [] # 문장의 각 단어가 정수형 인코딩 된 후 그 정수를 담을 빈 리스트
        
        seq = re.sub(CHANGE_FILTER, "", seq) # 불필요한 문자들을 제거하고
        
        # 문장 안에 있는 각 단어에 대한 정수형 인코딩 시작
        for word in seq.split(): # 문장을 띄어쓰기 단위로 생성한 리스트의 각 단어들을 순회하면서
            if dictionary.get(word) is not None: # 만약 그 단어가 사전에 존재하는 단어라면
                seq_idx.append(dictionary[word]) # 사전에 할당되어 있는 숫자를 seq_idx에 append하고
            else: # 없는 단어라면
                seq_idx.append(dictionary[OOV]) # unknown 토큰을 append한다.
        # 문장에 대한 정수형 인코딩 끝

        # zero padding 시작
        if len(seq_idx) > MAX_LEN: # 만약 seq_idx의 길이가 MAX_LEN보다 길다면
            seq_idx = seq_idx[:MAX_LEN] # 최대 길이에서 자른다.
        
        # encoder input, encoder output, encoder target의 상황에 따라, 
        if position == "encoder": # encoder용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx))*[dictionary[PAD]] # 최대 길이 - index의 길이만큼 0으로 이뤄진 패딩 리스트를 생성한다.
        
            if padding == "pre": # pre padding이라면
                temp_list.extend(zero_padding) # zero padding값을 먼저 extend하고
                temp_list.extend(seq_idx)
            else: # 아니라면
                temp_list.extend(seq_idx)
                temp_list.extend(zero_padding) # zero padding 값을 나중에 extend한다.
        elif position == "decoder output": # decoder output용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx) - 1)*[dictionary[PAD]] # SOS토큰을 고려해서 1을 추가적으로 빼준다.
        
            if padding == "pre": # pre-padding 이라면
                temp_list.append(dictionary[SOS]) # SOS토큰을 먼저 append 하고
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend한다.
            else: # 아니라면
                temp_list.append(dictionary[SOS]) # SOS토큰을 먼저 append 하고
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend한 후
                temp_list.extend(zero_padding) # zero padding 값을 extend 한다.
        else: # decoder target용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx) - 1)*[dictionary[PAD]] # EOS토큰을 고려해서 1을 추가적으로 빼준다.
        
            if padding == "pre":  #pre-padding이라면
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend 하고
                temp_list.append(dictionary[EOS]) # EOS토큰을 append한다.
            else: # 아니라면
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend 하고
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.append(dictionary[EOS]) # EOS토큰을 append한다.
            
            
        padded_sequence.append(temp_list) # 패딩이 완료된 각 요소는 전체 요소를 담을 list에 append한다.
    
    return np.array(padded_sequence)

In [15]:
encoder_input = make_sequence(inputs, char2idx, position="encoder")
decoder_output = make_sequence(inputs, char2idx, position="decoder output")
decoder_target = make_sequence(inputs, char2idx, position="decoder target")

In [16]:
encoder_input.shape

(11823, 76)

In [17]:
data_configs = {
    "char2idx" : char2idx,
    "idx2char" : idx2char,
    'vocab_size' : len(char2idx),
    'pad_symbol' : PAD,
    'sos_symbol' : SOS,
    'eos_symbol' : EOS,
    'ovv_symbol' : OOV
}

In [18]:
TRAIN_INPUTS = "./Chatbot_data/train_inputs.npy"
TRAIN_OUTPUTS = "./Chatbot_data/train_outputs.npy"
TRAIN_TARGETS = "./Chatbot_data/train_targets.npy"
DATA_CONFIGS = "./Chatbot_data/data_config.json"

In [19]:
np.save(TRAIN_INPUTS, encoder_input)
np.save(TRAIN_OUTPUTS, decoder_output)
np.save(TRAIN_TARGETS, decoder_target)

In [20]:
json.dump(data_configs, open(DATA_CONFIGS, 'w'))